# Alternating Least Squares

## 1. Import and explore data

In [72]:
import pandas as pd

df = pd.read_csv("data/train_2_long.csv")
print(f"Dataframe shape: {df.shape}")
df.info(show_counts=True)

Dataframe shape: (2380730, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380730 entries, 0 to 2380729
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2380730 non-null  int64
 1   item_id  2380730 non-null  int64
dtypes: int64(2)
memory usage: 36.3 MB


In [73]:
len_before = df.shape[0]
print(f"Number of entries before dropping duplicates: {len_before}")
df = df.drop_duplicates()
len_after = df.shape[0]
print(f"Number of entries after dropping duplicates: {len_after}")

Number of entries before dropping duplicates: 2380730
Number of entries after dropping duplicates: 2380730


In [74]:
print(f"There are {df['user_id'].nunique()} unique users.")
print(f"There are {df['item_id'].nunique()} unique items.")

There are 52643 unique users.
There are 91599 unique items.


In [75]:
user_interactions = df["user_id"].value_counts()
print(f"The average user interacted with {int(user_interactions.mean())} items.\n")
print(f"User interactions stats:\n{user_interactions.describe()}")

The average user interacted with 45 items.

User interactions stats:
count    52643.000000
mean        45.224056
std         77.958253
min         16.000000
25%         19.000000
50%         26.000000
75%         45.000000
max      10682.000000
Name: count, dtype: float64


In [76]:
item_interactions = df["item_id"].value_counts()
print(f"The average item has {int(item_interactions.mean())} interactions.\n")
print(f"User interactions stats:\n{item_interactions.describe()}")

The average item has 25 interactions.

User interactions stats:
count    91599.000000
mean        25.990786
std         38.397318
min          1.000000
25%         10.000000
50%         15.000000
75%         28.000000
max       1741.000000
Name: count, dtype: float64


In [77]:
most_active_user = user_interactions.index[0]
df[df["user_id"] == most_active_user]

,user_id,item_id
18999,150,9566
19000,150,60
19001,150,5357
19002,150,13171
19003,150,13172
...,...,...
29676,150,21470
29677,150,21471
29678,150,552
29679,150,21472


## 2. Prepare the data

In [ ]:
# Add implicit value = 1.0 (positive interaction)
# df["value"] = 1.0

In [78]:
# Check new df
df.head()

,user_id,item_id
0,0,28261
1,0,388
2,0,5731
3,0,401
4,0,28284


### 2.1. Encoding user and items
`implicit` requires zero-based IDs

In [79]:
# Check ID ranges
print(f"User ID min: {df['user_id'].min()}, max: {df['user_id'].max()}")
print(f"Item ID min: {df['item_id'].min()}, max: {df['item_id'].max()}")

User ID min: 0, max: 52642
Item ID min: 6, max: 91604


In [80]:
# Map IDs to zero-based
user_map = {uid: u_idx for u_idx, uid in enumerate(sorted(df["user_id"].unique()))}
item_map = {iid: i_idx for i_idx, iid in enumerate(sorted(df["item_id"].unique()))}

### 2.2. Function to convert to sparse matrices

In [84]:
from scipy.sparse import coo_matrix

def df_to_sparse(df):
    """ Function that converts a dataframe to a sparse matrix. """
    rows = df["user_id"].map(user_map)
    cols = df["item_id"].map(item_map)
    vals = np.ones(len(df))

    matrix = coo_matrix((vals, (rows, cols)),
                        shape=(len(user_map), len(item_map)))
    
    return matrix

## 3. Split data into training and testing
using `sklearn.model selection.train_test_split`

* Split user-item interactions into training and testing.
* Ensures a more realistic evaluation when generalizing to new users or items.

For every user:
* Perform `train_test_split` on the items it has interacted with.

In [94]:
from sklearn.model_selection import train_test_split

# Define group
user_groups = df.groupby("user_id")

# Store splits
train_pairs, test_pairs = [], []

# Iterate through groups
for user, items in user_groups:
    train, test = train_test_split(items, test_size=0.2, random_state=17)
    train_pairs.append(train)
    test_pairs.append(test)

In [100]:
# Turn lists into dataframes
train_df = pd.concat(train_pairs)
test_df = pd.concat(test_pairs)

# Check dataframe shapes
print(train_df.shape, test_df.shape)

# Make sure split is done properly
print(train_df.shape[0] + test_df.shape[0] == df.shape[0])

(1881725, 2) (499005, 2)
True


## 4. Train ALS model
Alternating Least Squares (ALS) is a matrix factorization algorithm which typically works well for implicit feedback data, such as user-item interactions, as seen in this [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf) paper.

### 4.1. Create csr_matrix representations for train and test set

In [104]:
# Training matrix
train_csr = df_to_sparse(train_df).tocsr()
train_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1881725 stored elements and shape (52643, 91599)>

In [109]:
# Test matrix
test_csr = df_to_sparse(test_df).tocsr()
test_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 499005 stored elements and shape (52643, 91599)>

### 4.2. Train model
Hyperparameters used for initial training:
* `factors` = 50: the number of latent factors in for decomposition
* `regularization` = 0.1: the regularization parameter ($\lambda$)
* `iterations` = 20: the number of training iterations
* `alpha` = 1.0: the weights given for positive interactions
* `use_cg` = True: used a faster solver



In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(
    factors=50,             # hyperparameter (latent factors)
    regularization=0.1,     # hyperparameter ()
    iterations=20,          # hyperparameter (epochs)
    alpha=1.0               # hyperparameter (alpha)
    use_cg=True,
    calculate_training_loss=True
)

model.fit(train_csr, show_progress=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Check matrix decomposition shapes
print("User factors shape:", model.user_factors.shape)
print("Item factors shape:", model.item_factors.shape)

User factors shape: (52643, 100)
Item factors shape: (91599, 100)


## 5. Recommend top 20 items for users in test set

In [221]:
K = 20 # top 20, NDCG@K

recommended_items = model.recommend_all(train_csr, 
                                        N=K,
                                        filter_already_liked_items=True)

In [222]:
# Model recommended top 20 items for ALL users.
len(recommended_items) == df["user_id"].nunique()

True

## 6. Evaluate with NDCG@20

In [223]:
import numpy as np
from sklearn.metrics import ndcg_score

def batched_ndcg_at_k(recommended_items, test_matrix, k=20, batch_size=1000):
    num_users = test_matrix.shape[0]
    ndcg_scores = []

    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)

        # Extract true relevance for the batch
        true_batch = test_matrix[start:end].toarray()

        # Build predicted score matrix for this batch
        pred_batch = np.zeros_like(true_batch)
        for i, user_idx in enumerate(range(start, end)):
            items = recommended_items[user_idx]
            pred_batch[i, items[:k]] = 1.0 / (np.arange(1, k + 1))  # rank weighting

        # Compute vectorized NDCG for the batch
        batch_ndcg = ndcg_score(true_batch, pred_batch, k=k)
        ndcg_scores.append(batch_ndcg)

    return np.mean(ndcg_scores)

# Example usage
mean_ndcg = batched_ndcg_at_k(recommended_items, test_csr, k=K, batch_size=1000)
print(f"Mean NDCG@20 = {mean_ndcg:.4f}")


Mean NDCG@20 = 0.0556


In [ ]:
# from sklearn.metrics import ndcg_score
# import numpy as np

# # Store NCDG@20 scores for each user in the test set
# ndcg_scores = []

# for user_idx in range(test_csr.shape[0]):
#     true_items = test_csr[user_idx].toarray().ravel()

#     if len(true_items) == 0:
#         print(f"Skipping user index: {user_idx}")
#         continue

#     # Predicted scores
#     pred_scores = np.zeros_like(true_items)
#     pred_scores[recommended_items[user_idx]] = 1.0 / (np.arange(1, 21)) # weighted ranking


#     ndcg = ndcg_score([true_items], [pred_scores], k=K)
#     ndcg_scores.append(ndcg)

# print(f"Average NDCG Score: {np.mean(ndcg_scores): 0.5f}")


In [174]:
recommended_items[user_idx]

array([  13,  114,  527, 4938,  387,  432, 2374,  373,  673,  975,  379,
         11,  136, 4956,  374, 5964, 1364,   90,  447, 5182], dtype=int32)

In [175]:
np.where(true_items == 1)

(array([  395,  5725, 22030, 29775]),)

## 7. Write output file

### 7.1. Create reverse maps for user_idx and item_idx

In [143]:
reverse_user_map = {user_idx: user_id for user_id, user_idx in user_map.items()}
reverse_item_map = {item_idx: item_id for item_id, item_idx in item_map.items()}

In [189]:
output_file = "als100f_50e.txt"

with open(output_file, "w") as f:
    for user_idx, item_indices in enumerate(recommended_items):
        user_id = reverse_user_map[user_idx]
        item_ids = [str(reverse_item_map[i]) for i in item_indices]
        f.write(f"{user_id} {' '.join(item_ids)}\n")


In [ ]:
# Map all item_idx back to original item_id
[reverse_item_map.get(item, item) for item in recommended_items[0]] 

[19,
 120,
 533,
 4944,
 393,
 438,
 2380,
 379,
 679,
 981,
 385,
 17,
 142,
 4962,
 380,
 5970,
 1370,
 96,
 453,
 5188]